In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a Prophet Model using Vertex AI Tabular Workflows

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/official/automl/prophet_on_vertex_pipelines.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/official/automl/prophet_on_vertex_pipelines.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/prophet_on_vertex_pipelines.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview

In this tutorial, you take on the role of a store planner who must determine how much inventory they will need to order for each of their products and stores for November 2019. You accomplish this by fitting a Prophet model to your historical sales data using Vertex AI Tabular Workflows.

Like [BQML ARIMA+](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series), Meta's [Prophet](https://facebook.github.io/prophet/docs/quick_start.html) attempts to decompose each time series into trends, seasons, and holidays, producing a forecast using the aggregation of these models' predictions. One of many differences, however, is that BQML ARIMA+ uses ARIMA to model the trend component, while Prophet attempts to fit a curve using a piecewise logistic or linear model. See the [paper](https://peerj.com/preprints/3190/) for details.

Prophet on Vertex AI Tabular Workflows is a Vertex Pipeline that fits Prophet models to your data. On top of Prophet, a few additional features are supported by this pipeline:
  - Support for multiple time series. This is accomplished by using a Dataflow job to train one Prophet model for each time series in the given data.
  - AutoML Tables-style splitting and windowing strategies.
  - Reading data from either BigQuery tables or CSVs stored in Google Cloud Storage (GCS). The format for each row is the same as what Vertex AI Forecasting expects.
  - Hyperparameter tuning using grid search and Prophet's built-in backtesting logic.

### Objective

In this tutorial, you learn how to create several Prophet models using a training Vertex AI Pipeline from Google Cloud Pipeline Components (GCPC), and then do a batch prediction using the corresponding prediction pipeline.

This tutorial uses the following Google Cloud ML services and resources:

- BigQuery
- Cloud Storage
- Vertex AI
- Dataflow

The steps performed are:

1. Train the Prophet models.
1. View the evaluation metrics.
1. Make a batch prediction with the Prophet models.


### Dataset

This tutorial will use a synthetic dataset where product sales are dependent on a variety of factors such as advertisements, holidays, and locations. Prophet will only be using historical sales, so it should not be trivial to get decent evaluation metrics.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* BigQuery
* Dataflow

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), [Dataflow
pricing](https://cloud.google.com/dataflow/pricing), and [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements. You need the following:

- The Cloud Storage SDK
- Python 3
- virtualenv
- Jupyter notebook running in a virtual environment with Python 3

The Cloud Storage guide to [Setting up a Python development environment](https://cloud.google.com/python/setup) and the [Jupyter installation guide](https://jupyter.org/install) provide detailed instructions for meeting these requirements. The following steps provide a condensed set of instructions:

1. [Install and initialize the SDK](https://cloud.google.com/sdk/docs/).

2. [Install Python 3](https://cloud.google.com/python/setup#installing_python).

3. [Install virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) and create a virtual environment that uses Python 3.  Activate the virtual environment.

4. To install Jupyter, run `pip3 install jupyter` on the command-line in a terminal shell.

5. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

6. Open this notebook in the Jupyter Notebook Dashboard.


## Install additional packages

Install the latest version of the Google Cloud Pipeline Components (GCPC) SDK.

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

# TODO: Install GCPC from a release once prophet has been included in a release.
! git clone https://github.com/kubeflow/pipelines.git
! (pip3 install --upgrade $USER_FLAG \
    google-cloud-bigquery[pandas]==2.34.4 \
    google-cloud-aiplatform==1.18.3 \
    pipelines/components/google-cloud/.)

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

**Note: You may get a message saying "Your session crashed for an unknown reason.", this is expected. Once this cell has finished running, continue on. You do not need to re-run any of the cells above.**

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU runtime

This tutorial does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the following APIs: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. If you are running this notebook locally, you will need to install the [Cloud SDK]((https://cloud.google.com/sdk)).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

6. (optional) You may also specify a service account to use to run Vertex Pipelines in the project.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

### Set your project ID

Set your project ID below. If you know know your project ID, leave the field blank and the following cells may be able to find it. Optionally, you may also set a service account in the cell below.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

print(PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region
All BigQuery operations (`DATA_REGION`) are set to run in the `US` multi-region. This is required by the training pipeline because the data you will be using is stored in this region. All destination tables will also be stored in this region.

You may change the `REGION` variable, which is used for Vertex Forecasting operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}
DATA_REGION = "US"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

- In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

- **Click Create service account**.

- In the **Service account name** field, enter a name, and click **Create**.

- In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex" into the filter box, and select **Vertex Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

- Click Create. A JSON file that contains your key downloads to your local environment.

- Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS '[your-service-account-key-path]'

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

Create the bucket if it doesn't already exist.

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + UUID

! gsutil ls -b $BUCKET_URI || gsutil mb -l $REGION $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Service Account
You use a service account to create Vertex AI Pipeline jobs.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines
Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step. You only need to run this step once per service account.

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/dataflow.developer

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import json
import urllib

from google.cloud import aiplatform, bigquery
from google_cloud_pipeline_components.experimental.automl.forecasting import \
    utils

## Initialize Vertex SDK for Python

Initialize the Vertex SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Define train and prediction data

### Location of BigQuery destination table.

#### Create two datasets, one for each model you train. To make things simpler, create the datasets in the same region as the training data.

In [ ]:
dataset_name = f"forecasting_demo_prophet_{UUID}"

dataset_path = ".".join([PROJECT_ID, dataset_name])

# Must be same region as TRAINING_DATASET_BQ_PATH.
client = bigquery.Client(project=PROJECT_ID)
bq_dataset = bigquery.Dataset(dataset_path)
bq_dataset.location = DATA_REGION
bq_dataset = client.create_dataset(bq_dataset)
print(f"Created bigquery dataset {dataset_path} in {DATA_REGION}")

### Location of BigQuery training data.

Before training a model, you must first generate our dataset of store sales. This dataset will include multiple products and stores, and it will also simulate factors such as advertisements and holiday effects. The data will be split into `TRAIN`, `VALIDATE`, `TEST`, and `PREDICT` sets, where the last three sets are all 1 month in duration.

#### Begin by defining the subqueries that will create this base sales data.

In [ ]:
base_data_query = """
  WITH

    -- Create time series for each product + store with some covariates.
    time_series AS (
      SELECT
        CONCAT("id_", store_id, "_", product_id) AS id,
        CONCAT('store_', store_id) AS store,
        CONCAT('product_', product_id) AS product,
        date,
        -- Advertise 1/100 products.
        IF(
          ABS(MOD(FARM_FINGERPRINT(CONCAT(product_id, date)), 100)) = 0,
          1,
          0
        ) AS advertisement,
        -- Mark Thanksgiving sales as holiday sales.
        IF(
          EXTRACT(DAYOFWEEK FROM date) = 6
            AND EXTRACT(MONTH FROM date) = 11
            AND EXTRACT(DAY FROM date) BETWEEN 23 AND 29,
          1,
          0
        ) AS holiday,
        -- Set when each data split ends.
        CASE
          WHEN date < '2019-09-01' THEN 'TRAIN'
          WHEN date < '2019-10-01' THEN 'VALIDATE'
          WHEN date < '2019-11-01' THEN 'TEST'
          ELSE 'PREDICT'
        END AS split,
      -- Generate the sales with one SKU per date.
      FROM
        UNNEST(GENERATE_DATE_ARRAY('2017-01-01', '2019-12-01')) AS date
      CROSS JOIN
        UNNEST(GENERATE_ARRAY(0, 10)) AS product_id
      CROSS JOIN
        UNNEST(GENERATE_ARRAY(0, 3)) AS store_id
    ),

    -- Randomly determine factors that contribute to how syntheic sales are calculated.
    time_series_sales_factors AS (
      SELECT
        *,
        ABS(MOD(FARM_FINGERPRINT(product), 10)) AS product_factor,
        ABS(MOD(FARM_FINGERPRINT(store), 10)) AS store_factor,
        [1.6, 0.6, 0.8, 1.0, 1.2, 1.8, 2.0][
          ORDINAL(EXTRACT(DAYOFWEEK FROM date))] AS day_of_week_factor,
        1 +  SIN(EXTRACT(MONTH FROM date) * 2.0 * 3.14 / 24.0) AS month_factor,
        -- Advertised products have increased sales factors for 5 days.
        CASE
          WHEN LAG(advertisement, 0) OVER w = 1.0 THEN 1.2
          WHEN LAG(advertisement, 1) OVER w = 1.0 THEN 1.8
          WHEN LAG(advertisement, 2) OVER w = 1.0 THEN 2.4
          WHEN LAG(advertisement, 3) OVER w = 1.0 THEN 3.0
          WHEN LAG(advertisement, 4) OVER w = 1.0 THEN 1.4
          ELSE 1.0
        END AS advertisement_factor,
        IF(holiday = 1.0, 2.0, 1.0) AS holiday_factor,
        0.001 * ABS(MOD(FARM_FINGERPRINT(CONCAT(product, store, date)), 100)) AS noise_factor
      FROM
        time_series
      WINDOW w AS (PARTITION BY id ORDER BY date)
    ),

    -- Use factors to calculate synthetic sales for each time series.
    base_data AS (
      SELECT
        id,
        store,
        product,
        date,
        split,
        advertisement,
        holiday,
        (
          (1 + store_factor)
          * (1 + product_factor)
          * (1 + month_factor + day_of_week_factor)
          * (
            1.0
            + 2.0 * advertisement_factor
            + 3.0 * holiday_factor
            + 5.0 * noise_factor
          )
        ) AS sales
      FROM
        time_series_sales_factors
      )
"""

Next, convert this base sales data into a dataset you use to train a model, and a dataset you pass to a trained model at serving time. The training dataset will include the `TRAIN`, `VALIDATE`, and `TEST` splits, while the prediction dataset will include the `PREDICT` split and also the `TEST` split to provide context information.

In [ ]:
TRAINING_DATASET_BQ_PATH = f"bq://{dataset_path}.train"
PREDICTION_DATASET_BQ_PATH = f"bq://{dataset_path}.pred"

train_query = f"""
    CREATE OR REPLACE TABLE `{dataset_path}.train` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split != 'PREDICT'
"""
client.query(train_query).result()
print(f"Created {TRAINING_DATASET_BQ_PATH}.")

pred_query = f"""
    CREATE OR REPLACE TABLE `{dataset_path}.pred` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split = 'TEST'

    UNION ALL

    SELECT * EXCEPT (sales), NULL AS sales
    FROM base_data
    WHERE split = 'PREDICT'
"""
client.query(pred_query).result()
print(f"Created {PREDICTION_DATASET_BQ_PATH}.")

You can take a look at the sales data that was generated. Later in this tutorial, we will visualize the time series along with our forecast.

The model is trained with data from January 2017 to October 2019 inclusive.

#### Look at the training data

In [ ]:
query = f"SELECT * FROM `{dataset_path}.train` LIMIT 10"
client.query(query).to_dataframe().head()

The table used for prediction contains data from November 2019. It also includes actuals from October 2019 as context information.

#### Look at the prediction data

In [ ]:
query = f"SELECT * FROM `{dataset_path}.pred` LIMIT 10"
client.query(query).to_dataframe().head()

## Create a Prophet model

Use GCPC to get the Prophet pipeline to start an `aiplatform.PipelineJob`.

### Create the training job

In [ ]:
time_column = "date"  # @param {type: "string"}
time_series_identifier_column = "id"  # @param {type: "string"}
target_column = "sales"  # @param {type: "string"}
forecast_horizon = 30  # @param {type: "integer"}
optimization_objective = "rmse"  # @param {type: "string"}
data_granularity_unit = "day"  # @param {type: "string"}
split_column = "split"  # @param {type: "string"}
window_stride_length = 1  # @param {type: "integer"}

(
    train_job_spec_path,
    train_parameter_values,
) = utils.get_prophet_train_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    root_dir=os.path.join(BUCKET_URI, "pipeline_root"),
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    target_column=target_column,
    forecast_horizon=forecast_horizon,
    optimization_objective=optimization_objective,
    data_granularity_unit=data_granularity_unit,
    predefined_split_key=split_column,
    data_source_bigquery_table_path=TRAINING_DATASET_BQ_PATH,
    window_stride_length=window_stride_length,
    trainer_dataflow_max_num_workers=50,
    max_num_trials=2,
    trainer_service_account=SERVICE_ACCOUNT,
    dataflow_service_account=SERVICE_ACCOUNT,
)

### Run the training pipeline

Use the Vertex AI Python SDK to kick off a training pipeline run. Once the run has started, the following cell outputs a link that will allow you to monitor the run. The link should look like this:

`https://console.cloud.google.com/vertex-ai/locations/[REGION]/pipelines/runs/[DISPLAY_NAME]`

In [ ]:
# The display name should be unique even if this cell is rerun.
DISPLAY_NAME = f"forecasting-demo-train-{generate_uuid()}"

job = aiplatform.PipelineJob(
    job_id=DISPLAY_NAME,
    display_name=DISPLAY_NAME,
    pipeline_root=os.path.join(BUCKET_URI, DISPLAY_NAME),
    template_path=train_job_spec_path,
    parameter_values=train_parameter_values,
)
job.run(service_account=SERVICE_ACCOUNT)

If you ever want to reuse an existing run, the above command can be replaced with:

```
job = aiplatform.PipelineJob.get('projects/[PROJECT_NUMBER]/locations/[REGION]/pipelineJobs/[PIPELINE_RUN_NAME]')

```

## Review model evaluation scores
After your model has finished training, you can review the evaluation scores for it.

#### Metrics are always reported via the `metrics` table in the destination dataset.

In [ ]:
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "model-evaluation":
        metrics = task_detail.outputs["evaluation_metrics"].artifacts[0].metadata
        break
else:
    raise ValueError("Couldn't find the model evaluation task.")

print("Evaluation metrics:\n")
dict(metrics)

## Create and run prediction job

### Create prediction job

Like with the training job, get the prediction pipeline from GCPC and use it to create an `aiplatform.PipelineJob`.

In [ ]:
# Get the model name programmatically, you can find this by looking at the
# execution graph in Vertex AI Pipelines.
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "model-upload":
        model = task_detail.outputs["model"].artifacts[0].metadata["resourceName"]
        break
else:
    raise ValueError("Couldn't find the model training task.")

# Use the model when creating the pipeline parameters.
(
    prediction_job_spec_path,
    prediction_parameter_values,
) = utils.get_prophet_prediction_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    model_name=model,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    target_column=target_column,
    data_source_bigquery_table_path=TRAINING_DATASET_BQ_PATH,
    bigquery_destination_uri=dataset_path,
    max_num_workers=50,
)

### Run the prediction pipeline

Use the Vertex AI Python SDK to kick off a prediction pipeline run. Once the run has started, the following cell outputs a link that will allow you to monitor the run. The link should look like this:

`https://console.cloud.google.com/vertex-ai/locations/[REGION]/pipelines/runs/[DISPLAY_NAME]`

In [ ]:
# The display name should be unique even if this cell is rerun.
DISPLAY_NAME = f"forecasting-demo-predict-{generate_uuid()}"

job = aiplatform.PipelineJob(
    job_id=DISPLAY_NAME,
    display_name=DISPLAY_NAME,
    pipeline_root=os.path.join(BUCKET_URI, DISPLAY_NAME),
    template_path=prediction_job_spec_path,
    parameter_values=prediction_parameter_values,
)
job.run(service_account=SERVICE_ACCOUNT)

### Get the predictions

Next, get the results from the completed batch prediction job. These are always written to a table called `predictions` under the output dataset.

In [ ]:
# Get the prediction table programmatically, you can find this by looking at the
# execution graph in Vertex AI Pipelines.
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "bigquery-query-job-2":
        pred_table = (
            task_detail.outputs["destination_table"].artifacts[0].metadata["tableId"]
        )
        break
else:
    raise ValueError("Couldn't find the prediction task.")

query = f"SELECT * FROM `{dataset_path}.{pred_table}`"
preds = client.query(query).to_dataframe()
preds.head()

## Visualize the forecasts

Lastly, follow the given link to visualize the generated forecasts in [Data Studio](https://support.google.com/datastudio/answer/6283323?hl=en).
The code block included in this section dynamically generates a Data Studio link that specifies the template, the location of the forecasts, and the query to generate the chart. The data is populated from the forecasts generated earlier.

You can inspect the used template at https://datastudio.google.com/c/u/0/reporting/067f70d2-8cd6-4a4c-a099-292acd1053e8. This was created by Google specifically to view forecasting predictions.

In [ ]:
def _sanitize_bq_uri(bq_uri: str):
    if bq_uri.startswith("bq://"):
        bq_uri = bq_uri[5:]
    return bq_uri.replace(":", ".")


def get_data_studio_link(
    batch_prediction_bq_input_uri: str,
    batch_prediction_bq_output_uri: str,
    time_column: str,
    time_series_identifier_column: str,
    target_column: str,
):
    """Creates a link that fills in the demo Data Studio template."""
    batch_prediction_bq_input_uri = _sanitize_bq_uri(batch_prediction_bq_input_uri)
    batch_prediction_bq_output_uri = _sanitize_bq_uri(batch_prediction_bq_output_uri)
    query = f"""
        SELECT
          CAST(input.{time_column} as DATETIME) timestamp_col,
          CAST(input.{time_series_identifier_column} as STRING) time_series_identifier_col,
          CAST(input.{target_column} as NUMERIC) historical_values,
          CAST(predicted_{target_column}.value as NUMERIC) predicted_values,
        FROM `{batch_prediction_bq_input_uri}` input
        LEFT JOIN `{batch_prediction_bq_output_uri}` output
          ON
            TIMESTAMP(input.{time_column}) = TIMESTAMP(output.{time_column})
            AND CAST(input.{time_series_identifier_column} as STRING) = CAST(
              output.{time_series_identifier_column} as STRING)
    """
    params = {
        "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
        "ds0.connector": "BIG_QUERY",
        "ds0.projectId": PROJECT_ID,
        "ds0.billingProjectId": PROJECT_ID,
        "ds0.type": "CUSTOM_QUERY",
        "ds0.sql": query,
    }
    base_url = "https://datastudio.google.com/c/u/0/reporting"
    url_params = urllib.parse.urlencode({"params": json.dumps(params)})
    return f"{base_url}?{url_params}"

In [ ]:
actuals_table = f"{dataset_path}.actuals"
query = f"""
    CREATE OR REPLACE TABLE `{actuals_table}` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split != 'TRAIN'
"""
client.query(query).result()
print(f"Created {actuals_table}.")

In [ ]:
print("Click the link below to view ARIMA predictions:")
print(
    get_data_studio_link(
        batch_prediction_bq_input_uri=actuals_table,
        batch_prediction_bq_output_uri=f"{dataset_path}.{pred_table}",
        time_column=time_column,
        time_series_identifier_column=time_series_identifier_column,
        target_column=target_column,
    )
)

## Clean up Vertex AI and BigQuery resources

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Model
- Cloud Storage Bucket
- BigQuery tables

In [ ]:
# Delete the model
aiplatform.Model(model).delete()

# Delete output datasets
client.delete_dataset(dataset_path, delete_contents=True, not_found_ok=True)

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI